In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import random 
import re
from scipy import spatial
%matplotlib inline
import math 
from collections import Counter 
from textblob import TextBlob
import nltk
import itertools
from itertools import permutations 
from operator import itemgetter 
nltk.download('wordnet')
from scipy.spatial import distance
import sys
#importing stopwords using nltk library
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
#creating tokens
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')
nltk.download('brown')
nltk.download('averaged_perceptron_tagger')
import itertools as it
import time
import random
import pickle as pk
import multiprocessing
import ast

In [ ]:
#Univ 1 graduate courses
Univ1_dataset=pd.read_csv('Univ-1.csv')
Univ1_dataset.head()

In [ ]:
Univ1_dataset=Univ1_dataset.applymap(str)
#removing the non-ascii characters present in the course and prerequisite column on university dataset
Univ1_dataset['Course']=Univ1_dataset['Course'].str.replace('\xa0',' ')
Univ1_dataset['Prerequisite']=Univ1_dataset['Prerequisite'].str.replace('\xa0',' ')

#removing the quotes and backslash from the course column of the dataset
def quotes(text):
  text = text.strip('\"')
  text=text.strip()
  return text
Univ1_dataset['Course']=Univ1_dataset['Course'].apply(quotes)

#tokenizing the prerquisites

Univ1_dataset['Prerequisite'] = Univ1_dataset['Prerequisite'].str.split(',')

#removing extra spaces in the pre list
Univ1_dataset['Prerequisite'] = Univ1_dataset['Prerequisite'].apply(lambda x:[w.lstrip() for w in x])

In [ ]:
#creating courses list and prerequsites list from the Univ1_dataset
course=list(Univ1_dataset['Course'])
pre=list(Univ1_dataset['Prerequisite'])

In [ ]:
#removing the prereq which are not university-1 graduate courses

for i in range(len(pre)):
  list_1=pre[i]
  for index,j in enumerate(list_1):
    if j not in course:
      list_1[index]='nan'

#removing the 'nan' values
for i in range(len(pre)):
  list_1=pre[i]
  for index,j in enumerate(list_1):
    if j=='nan':
      list_1.remove('nan')
for i in range(len(pre)):
  list_1=pre[i]
  for index,j in enumerate(list_1):
    if j=='nan':
      list_1.remove('nan')
for i in range(len(pre)):
  list_1=pre[i]
  for index,j in enumerate(list_1):
    if j=='nan':
      list_1.remove('nan')

In [ ]:
#data dict represents key as course from university-1 graduate courses and its prerequisite as value
data_dict={}
for i in range(len(course)):
  data_dict[course[i]]=pre[i]

In [ ]:
#removing redundancies in the data dictionary
data_dict['BNFO 700B']=[]
data_dict['BNFO 701B']=[]
data_dict['IS 701B']=[]
data_dict['CS 735']=['CS 675', 'CS 634', 'CS 643', 'CS 644']
data_dict['CS 726']=[]
data_dict['CS 700B']=[]
data_dict['CS 635']=['CS 505','CS 510']
data_dict['CS 640']=['CS 540']
data_dict['CS 646']=['CS 645','CS 646','CS 696']
data_dict['CS 650']=['CS 510']
data_dict['CS 668']=['CS 610','CS 650']
data_dict['CS 696']=['CS 652','CS 656']
data_dict['CS 701C']=[]
data_dict['CS 725']=[]
data_dict['CS 790A']=['CS 791']
data_dict['CS 790B']=['CS 791']
data_dict['CS 790C']=['CS 791']
data_dict['IS 591']=[]
data_dict['IS 592']=[]
data_dict['IS 593']=[]
data_dict['IS 601']=[]
data_dict['IS 612']=[]
data_dict['IS 613']=[]
data_dict['IS 614']=[]
data_dict['IS 616']=[]
data_dict['IS 650']=[]
data_dict['IS 661']=[]
data_dict['IS 664']=[]
data_dict['IS 676']=['IS 663','CS 673']
data_dict['IS 677']=[]
data_dict['IS 682']=[]
data_dict['IS 683']=[]
data_dict['IS 686']=[]
data_dict['IS 698']=[]
data_dict['IS 700']=[]
data_dict['IS 700B']=[]
data_dict['IS 700C']=[]
data_dict['IS 701']=[]
data_dict['IS 701B']=[]
data_dict['IS 701C']=[]
data_dict['IS 725']=[]
data_dict['IS 764']=[]
data_dict['IS 766']=[]
data_dict['IS 776']=[]
data_dict['IS 785']=[]
data_dict['IS 786']=[]
data_dict['IS 790']=[]
data_dict['IS 790D']=[]
data_dict['IS 790E']=[]
data_dict['IS 790F']=[]
data_dict['IS 791']=[]
data_dict['IT 610']=[]
data_dict['IT 635']=[]

In [ ]:
#removing the nan value
del data_dict['nan']

In [ ]:
#loading  University-1 graduate courses csv file
Univ1_graduate_dataset=pd.read_csv('Univ1_graduate_courses.csv')

In [ ]:
#removing unicode characters from course
Univ1_graduate_dataset['course']=Univ1_graduate_dataset['course'].str.replace('¬†',' ')
Univ1_graduate_dataset.drop('Unnamed: 3',axis=1,inplace=True)

#cleansing of data
def clean_text(text):
    text = text.lower()
    return text

#Genrating the frequent words
def gen_freq(text):
    #to store the list of words
    word_list = []
    for tw_words in text:
        word_list.extend(tw_words)
    #Create word frequencies using word_list
    word_freq = pd.Series(word_list).value_counts()
    return word_freq

In [ ]:
#core courses list for ms in cyberscecurity
cyber_core_list=['CS 608','CS 645','CS 646','CS 647','CS 656','CS 696']
#elective courses list for ms in cybersecurity
cyber_elective_list=['CS 633','CS 634','CS 643','CS 648','CS 652','CS 660','CS 673','CS 678','CS 680','CS 684','CS 708','CS 755','IS 680','IS 681','IS 682','IS 687','IT 620','IT 640','ECE 636','MGMT 688','MGMT 691','CS 610','CS 630','CS 631']

In [ ]:
#complete courses list for cybersecurity
cyber_courses_list=cyber_core_list + cyber_elective_list
len(cyber_courses_list)

#to compile one dictionary  for core and elective course where 1 will denote course is core and 0 will denote elective
cyber_dict={}
for i in range(len(cyber_courses_list)):
  key=cyber_courses_list[i]
  if key in cyber_core_list:
    cyber_dict[key]=[1,0]
  else:
    cyber_dict[key]=[0,1]

In [ ]:
#creating dictionary for cybersecurity courses and prerequsite
def prereq_dict(data_dict,course_list):
  course_dict={}
  for i in range(len(course_list)):
    key=course_list[i]
    if key in data_dict:
      course_dict[key]=data_dict.get(key)
    else:
      course_dict[key]=[]
  return course_dict

#cybersecurity  dict with prerequisite information
cyber_prereq_dict=prereq_dict(data_dict,cyber_courses_list)

#Adding double brackets for courses with prereqs in 'and' condition and correcting other courses prereqs
cyber_prereq_dict['ECE 636']=['CS 656']
cyber_prereq_dict['CS 643']=['CS 656','CS 644']
cyber_prereq_dict['CS 646']=['CS 656']
cyber_prereq_dict['CS 647']=['CS 645','CS 646','CS 696']
cyber_prereq_dict['CS 708']=[['CS 608', 'CS 645', 'CS 696']]
cyber_prereq_dict['IT 620']=[]
cyber_prereq_dict['CS 648']=['CS 656']

In [ ]:
#compare the dataset with cyber courses list and extract them
cyber_df=Univ1_graduate_dataset.loc[Univ1_graduate_dataset['course'].isin(cyber_courses_list)]

#Add  the 3 missing courses from  cyber courses list in the cyber dataframe
listOfSeries = [pd.Series(['ECE 636','Computer Networking Laboratory','Prerequisites: ECE 637 or CS 656. This course provides students with hands on training regarding the design, troubleshooting, modeling and evaluation of computer networks. In this course, students are going to experiment in a real test-bed networking environment, and learn about network design and troubleshooting topics and tools such as: network addressing, Address Resolution Protocol (ARP), basic troubleshooting tools (e.g. ping, ICMP), IP routing (e,g, RIP), route discovery (e.g. traceroute), TCP and UDP, IP fragmentation and many others. Student will also be introduced to the network modeling and simulation, and they will have the opportunity to build some simple networking models using the OPNET modeling tool and perform simulations that will help them evaluate their design approaches and expected network performance.'], index=cyber_df.columns ) ,
                pd.Series(['MGMT 688','Information Technology Business and the law','Includes historical and constitutional foundations, crimes, and torts in cyberspace, virtual property (patents online, copyrights in digital information, trade secrets in cyberspace, and cybermarks), electronic commerce contracting, electronic commerce, electronic money and the law, and information technology and online infringement of rights of intellectual property.'], index=cyber_df.columns ) ,
                pd.Series(['MGMT 691','Legal and Ethical issues','Explores the legal and ethical responsibilities of managers. Analyzes extent to which shareholders should be allowed to exercise their legitimate economic, legal, and ethical claims on corporate managers; extent of regulation of a particular industry, individual rights of the employee and various corporate interests, and corporate responsibility to consumers, society, and conservation of natural resources and the environment.'], index=cyber_df.columns ) ]

In [ ]:
# Pass a list of series to the append() to add multiple rows
cyber_df = cyber_df.append(listOfSeries , ignore_index=True)

#to reset the index 
cyber_df=cyber_df.reset_index(drop=True)

#converting the course_description and course_name column data into string format
cyber_df['course description']=cyber_df['course description'].apply(str)
cyber_df['course_name']=cyber_df['course_name'].apply(str)

#applying the clean_text function to the course_name column
cyber_df['course_name']=cyber_df['course_name'].apply(clean_text)


#tokenizing the words from the text in the course_name column
def getting_nouns(text):
  text_blob_object=TextBlob(text)

  return text_blob_object.words


cyber_df['words']=cyber_df['course_name'].apply(getting_nouns)

In [ ]:
#removing stopwords using nltk library
cyber_df['words'] = cyber_df['words'].apply(lambda x:[item for item in x if item not in stop])

#converting the words column into a single list 
list_of_topics=cyber_df['words'].tolist()

#making a single list
merged=list(itertools.chain.from_iterable(list_of_topics))

#removing the duplicates from the list
cyber_topics=list(dict.fromkeys(merged))
print(*cyber_topics,sep=",")

In [ ]:
cyber_df.info()

In [ ]:
len(cyber_courses_list)

In [ ]:
edges=[]
for course in cyber_courses_list:
  #print(course)
  if (cyber_prereq_dict[course]==[]):
    CourseId=cyber_courses_list[:]
    CourseId.remove(course)
    #print(CourseId)
    for i in CourseId:
      # print(course , (i,course))
      edges.append((i,course))
  else:
      if (len(cyber_prereq_dict[course])!=1 or type(cyber_prereq_dict[course][0])==str):
        for j in cyber_prereq_dict[course]:
          if(j== [] or j==course):
            continue
          else:
            # print(course , (j,course))
            edges.append((j, course)) 
      else:
        l = list(permutations(cyber_prereq_dict[course][0])) 
        for k in l:
          m=0
          while (m in range(len(k)-1)):
            if ((k[m],k[m+1]) in edges)==False:
              edges.append((k[m],k[m+1]))
            m+=1
          if ((k[len(k)-1],course) in edges)==False:   
            edges.append((k[len(k)-1],course))



for course in cyber_courses_list:
  if(cyber_prereq_dict[course]==[]):
    edges.append(('initial' , course))
# print(CourseId)
# print(edges)
       
G = nx.DiGraph()
G.add_edges_from(edges) 
pos = nx.spring_layout(G)
nx.draw_networkx_nodes(G,pos, cmap=plt.get_cmap('jet'),node_size = 1000,node_color="lightblue")
nx.draw_networkx_edges(G,pos,edge_color='b', edge_cmap=plt.cm.Blues,arrows=True,arrowstyle="->",arrowsize=10)
nx.draw_networkx_labels(G,pos)

plt.show()
#pylab.show()
print(len(edges))

In [ ]:
#creating a movie id dictionary with key as course and node number as value
movies = list(G.nodes())
movie_ids={}
for i,node in enumerate(G.nodes()):
  movie_ids[node]=i

In [ ]:
#creating the topic dictonary with key as course name and topic
topic_dict={}
course_topics=list(cyber_df['words'])
for i in range(len(cyber_courses_list)):
  topic_dict[cyber_courses_list[i]]=course_topics[i]

In [ ]:
#printing the topic_dict 
for i,j in topic_dict.items():
  print('{} : {}'.format(i,j))

In [ ]:
#Creating the input sequence for omega
a=random.sample(cyber_core_list,6)
b=random.sample(cyber_elective_list,4)
#creating the shuffle function
def shuffle(a,b):
  list_of_sequences=[]
  sequence_list=[]
  i=0
  while i<1000:
    sequence_list=[i] + list(a + b)
    list_of_sequences.append(sequence_list)
    #to randomly select the 6 core courses from the list
    a=random.sample(a,6)
    #to randomly the select 4 elective courses from the list
    b=random.sample(cyber_elective_list,4)
    sequence_list=[]
    i=i+1
  return list_of_sequences

list_sequences=shuffle(a,b)

In [ ]:
# split the sequence of a user into two sequences one to add to the graph and the other to use it as ground truth for evaluation
def divide_test(test_user_sequence):
    user_id = test_user_sequence[0]
    items_in_sequence = test_user_sequence[1:]
    l = len(items_in_sequence)//2
    previous = items_in_sequence[:l]

    left = items_in_sequence[l:]

    return user_id , previous , left

def compute_edges(gra, sequence):
    #start_time = time.time()
    SG= nx.DiGraph()
    SG.add_nodes_from(sequence)

    x=gra.subgraph(sequence)

    for i in range(0,len(sequence)):
        for p in range(i, len(sequence)):
            if (sequence[i],sequence[p]) in x.edges():
                SG.add_edge(sequence[i],sequence[p], weight= x[sequence[i]][sequence[p]]['weight'])
    #print(' compute edges seconds', time.time() - start_time)
    return SG
    

def somme(arcs):
    l = list(arcs.edges())
    sum=0
    for tup in l:
        sum=sum + arcs[tup[0]][tup[1]]['weight']
    return sum

def utility_sum(gra,sequence):
    return (somme(compute_edges(gra,sequence)))

# For the conditional variant of OMEGA
def noeuds(E,S):   # E is a set of tuples (l), S is a list of items (nodes)

    a, b = zip(*E)
    S =[ i for i in S ]
    l = list(a)+list(b)
    y= list(set(l))
    z= list(set(y)-set(S))

    return z


def noeuds1(E):
    a,b = zip(*E)
    l = list(a)+list(b)
    y= list(set(l))
    return y

def Union(E,e):
    return E.union(e)

def Union_List(lst1, lst2): 
    final_list = list(set(lst1) | set(lst2)) 
    return len(final_list)

###### REORDER

def REORDER(gra,structure,E,S):  # Compute sequence of items from a set of  edges E according to a graph gra
    #np.random.seed(123)  # to have for every call of the function the same topological structure
    # works in  k.log(k) |A|=k

    A = noeuds(E,S)
    #start_time = time.time()
    b= sorted(A, key=lambda x: structure.index(x))
    #print(' REORDER seconds', time.time() - start_time)

    return b


def REORDER1(gra,structure,E):  # Compute sequence of items from a set of  edges E according to a graph gra
    #np.random.seed(123)  # to have for every call of the function the same topological structure
    # works in  k.log(k) |A|=k

    A = noeuds1(E)
    #start_time = time.time()
    b= sorted(A, key=lambda x: structure.index(x))
    #print(' REORDER seconds', time.time() - start_time)

    return b

# OMEGA
def OMEGA(gra,structure,  k,S):

    edges = set()
    arcs = set(gra.edges()) # set of (edges) tuples [ (0,1) , (1,2),....   ]
    #print(len(arcs))
    # dif = [x for x in arcs if x not in edges]
    dif = arcs-edges
    C=[]    #  implementing the set C and while condition
    for e in dif:
        #print(Union( edges,[e]))

        if len(noeuds(Union( edges,[e]) ,S)) <= k:
            C.append(e)
    #utilities.append (utility(gra, REORDER(gra,Union(edges,[e]))))


    while C :
        maximum = -1   #searching in C the edge that will maximizes the utility function
        for i in C:
            value  = utility_sum(gra, REORDER1(gra,structure, Union(edges,[i]) ) )
            #print('utility', value)
            if maximum < value:
                maximum= value
                tup= i # the edge to add with the maximum utility
        
        edges.add(tup)

#print(edges)
        if len(noeuds(edges,S )) ==k:
                return  REORDER( gra,structure, edges,S  )

        dif = arcs-edges
       
   #implementing the set C and while condition
        C=set()
        

        for e in dif:

            if len(noeuds(Union( edges,[e]),S )) <= k:
               C.add(e)

#print(C)
    return  REORDER( gra,structure, edges,S  )


def Precision_k(gra,structure, test_user_sequence, k):
    
    user_id, previous, left = divide_test(test_user_sequence)
    reco = OMEGA(gra,structure,k, previous)
    reco =[str(i) for i in reco]
    
    print(reco)
    intersection = list(set(reco) & set(left) )
    return (len (intersection)/k)



# Our Graph Creation for a sequence (  one user  )
def Create_OUR(test_user_sequence,z):

    transitions = pk.load(open('final_matrix2', 'rb'))
    user_id, previous, left = divide_test(test_user_sequence)
    candidate_items = list(set(movies).difference(previous))
    #candidate_items = left
    gra = nx.DiGraph()
    for j in previous:
        gra.add_node(j)

    for i in candidate_items:

        gra.add_node(i)
        gra.add_edge(i,i)
        index_i = movie_ids[i]
        gra[i][i]['weight'] = transitions[index_i, index_i]

    last_items = previous[-z:]
    for item in last_items:
        index_last = movie_ids[item]

        for i in candidate_items:
            index_item = movie_ids[i]
            gra.add_edge(item,i)
            gra[item][i]['weight'] = transitions[ index_last,index_item]

    return gra


def worker( argument, results):
    
    G= nx.DiGraph()

    G = Create_OUR(argument,3)

    #print(G.edges(data=True))

    toto = nx.DiGraph()

    toto = G.copy()

    toto.remove_edges_from(nx.selfloop_edges(toto,data=True))  # toto

    structure = list(nx.topological_sort(toto))  # toto
    
     
    results.append( Precision_k( G,structure,argument,5))


random.seed(123)
# In the paper, they used 500 sequences at random as a test set
# Split sequences into training and testing

def make_train_test (sequences, size):

    list_shuffled = list_sequences.copy()
    #random.shuffle(list_shuffled)
    train_data = list_shuffled[size:]
    test_data =list_shuffled[:size]
    return train_data, test_data



##########################

def proba(sequences, list_items,topic_dict):
    number_of_items = len(list_items)
    matrice = np.zeros(shape=(number_of_items,number_of_items))  # to store number of occurences of topics with respect to the threshold l
    matrice1 = np.zeros(shape=(number_of_items,number_of_items)) # to store number of occurences

    # Version of the probas without the parameter l
    for seq in sequences:
         items = seq[1:]
         for i in range(len(items)-1):
            for j in range(i+1, len(items)):
                
                index_i = list_items[items[i]]
                
                index_j = list_items[items[j]]
                matrice[index_i][index_j] += Union_List(topic_dict[items[i]],topic_dict[items[j]])
    
    return matrice

#Counting the occurance of an item in all the lists we have
def frequency (train, list_items,topic_dict):
    number_of_items = len(list_items)
    list_count = [0]* number_of_items

    for sequence in train:
        items = sequence[1:]
        for item in items:
          topic=0
          if item in list_items:
              index_item = list_items.index(item)
              topic=len(topic_dict[item]) 
              list_count[index_item]+=topic

    for i in range(len(list_count)):
        if list_count[i]<10:
            list_count[i]=0

    return list_count

# Dividing the occurance of each item by the number of lists we have in our set of sequences
def empirical_frequency_items(train, list_items,topic_dict):  # train: training set of sequences , items list of item ids

    number_of_items = len(list_items)
    list_count = [0]* number_of_items

    for sequence in train:
        items = sequence[1:]
        for item in items:
          topic=0
          if item in list_items:
            index_item = list_items.index(item)
            topic=len(topic_dict[item]) 
            list_count[index_item]+=1

    for i in range(len(list_count)):
        if list_count[i]<10:
            list_count[i]=0

    return [x/len(train) for x in list_count]


def Transition_matrix (list_sequences, movies):

    matrice= pk.load(open ('proba_matrix_without_l', 'rb'))
    # first we run Proba function to get the proba_matrix_without_l, we store it and loaod it for faster tests

    final_matrix = np.zeros(shape= (len(movies), len(movies)))

    frequencies = frequency(list_sequences, movies,topic_dict)
    emp_freq = empirical_frequency_items(list_sequences, movies,topic_dict)
    for i in range(len(movies)):
        for j in range(len(movies)):
            if i == j:
                final_matrix[i,i]= emp_freq[i]
                
            else :
                if frequencies[i] ==0:
                    final_matrix[i,j]= 0
                else:
                    final_matrix[i,j]= matrice[i,j]/frequencies[i]
    return final_matrix


#####################

#dividing the data into test and train

train_data, test_data = make_train_test(list_sequences,150)

m = proba(train_data, movie_ids,topic_dict)
print('part2')
pk.dump(m, open('proba_matrix_without_l','wb'))

print('start')
final_matrix = Transition_matrix(train_data, movies)
pk.dump(final_matrix , open('final_matrix2', 'wb')) 

start_time = time.time()
manager = multiprocessing.Manager()
results = manager.list()

jobs=[]
for i in range(150):
        p = multiprocessing.Process(target = worker, args=(test_data[i], results))
        jobs.append(p)
        p.start()

for proc in jobs:
    proc.join()
print ('sum(results)/len(results)',sum(results)/len(results))
print('whole', time.time() - start_time)

In [ ]:
#save the results from omega framework

In [ ]:
list_sequences

In [ ]:
#load the results obtained from the omega
data=open('omega_results_cyber.txt','r')

data_2=data.readlines()
results_sequences=[]
for i in range(150):
  line=data_2[i]
  line_1=ast.literal_eval(line)
  line_2=[i] + line_1
  line_3=line_2
  results_sequences.append(line_3)

In [ ]:
data_2

In [ ]:
len(results_sequences)

In [ ]:
#taking the corresponding sequence for the test results to combine
test_sequence=list_sequences[:150]

In [ ]:
test_sequence

In [ ]:
#creating final results sequence with previous sequence from the input sequence with sequence results obtained from the omega framework
final_sequence=[]
for i in range(len(test_sequence)):
  list_1=test_sequence[i]
  list_2=list_1[1:6]
  #print(list_2)
  list_3=results_sequences[i]
  list_4=list_3[1:]
  #print(list_4)
  final_sequence.append(list_2 + list_4) 

In [ ]:
print(*final_sequence,sep=",")

In [ ]:
#saving the final resulting sequences
with open('test_results_cyber.txt', 'w') as f:
    for item in final_sequence:
        f.write("%s\n" % item)